[View in Colaboratory](https://colab.research.google.com/github/GeniGaus/100DaysOfMLCode/blob/master/RNN_PredictNextWord.ipynb)

## LSTM RNN using Tensorflow

### Network structure
---
This network takes previous 3 words as input and predicts the next word.

![](https://www.codeproject.com/KB/AI/1237557/65a7c3f6-4110-4228-a605-11cb45dbf246.Png)

In [0]:
## setup data

import numpy as np
import collections

class DataHandler:
  
  # reads data and converts into array of tokens
  def read_data(self, text, isFile=False):
    if not isFile:
      words = text.split()
      words = np.array(words)
      words = np.reshape(words,[-1,])
      return words
  
  # creates a dictionary of words based on most common as appearing in data
  def build_dataset(self, text):
    count = collections.Counter(text).most_common()
    word_dict = dict()
    for word,_ in count:
      word_dict[word] = len(word_dict)
    word_dict_rev = dict(zip(word_dict.values(),word_dict.keys()))
    return word_dict, word_dict_rev

In [0]:
# test data

text = "In a sense , people are our proper occupation . Our job is to do them good and put up with them . But when they obstruct our proper tasks , they become irrelevant to us — like sun , wind , animals . Our actions may be impeded by them , but there can be no impeding our intentions or our dispositions . Because we can accommodate and adapt . The mind adapts and converts to its own purposes the obstacle to our acting . The impediment to action advances action . What stands in the way becomes the way . "

#dh = DataHandler()
#print(dh.read_data(text))
#dh.build_dataset(dh.read_data(text))

In [0]:
## create RNN with LSTM using tensorflow
import tensorflow as tf
from tensorflow.contrib import rnn

class RNNGenerator:
  
  def create_LSTM(self,inputs,weights,biases,seq_size,num_units):
    '''
    
    '''
    # converts input text into rows with seq_size(=3 here) tokens
    inputs = tf.reshape(inputs,[-1,seq_size])
    
    # converts each row with with seq_size(=3 here) columns
    inputs = tf.split(inputs, seq_size, 1)
    
    # creates LSTM with 2 layers
    rnn_model = rnn.MultiRNNCell([rnn.BasicLSTMCell(num_units),rnn.BasicLSTMCell(num_units)])
    
    # generates predictions
    outputs, states = rnn.static_rnn(rnn_model, inputs, dtype=tf.float32)
    
    return tf.matmul(outputs[-1],weights['out']) + biases['out']

------

Creates a tensorflow session to generate the environment in which  the model runs

In [0]:
import random
import numpy as np

class SessionRunner:
  
  training_iters = 50000
  
  def __init__(self,optimizer,accuracy,cost,lstm,initializer,writer):
    self.optimizer = optimizer
    self.accuracy = accuracy
    self.cost = cost
    self.lstm = lstm
    self.initializer = initializer
    self.writer = writer
  
  def run_session(self,x,y,n_input,dic,rev_dic,training_data):
    with tf.Session() as session:
      session.run(self.initializer)
      
      step = 0
      offset = random.randint(0,n_input+1)
      acc_total = 0 #total accuracy
      
      self.writer.add_graph(session.graph)
      
      while step < self.training_iters:
        if offset > (len(training_data) - n_input - 1):
          offset = random.randint(0,n_input+1)
        
        # get indexes of particular token
        sym_in_keys = [dic[str(training_data[i])] for i in range(offset,offset+n_input)]
        sym_in_keys = np.reshape(sym_in_keys, [-1,n_input,1])
        
        # create one hot encoding of the output
        sym_out_onehot = np.zeros([len(dic)], dtype=float)
        sym_out_onehot[dic[str(training_data[offset+n_input])]] = 1.0
        sym_out_onehot = np.reshape(sym_out_onehot, [1,-1])
        
        _, acc, loss, onehot_pred = session.run([self.optimizer,self.accuracy,self.cost,self.lstm], feed_dict={x: sym_in_keys, y: sym_out_onehot})
        acc_total += acc
        
        ## see training output per 1000 training
        if(step+1)%1000 == 0:
          print("Iteration = " + str(step + 1) + ", Average Accuracy= " + "{:.2f}%".format(100*acc_total/1000))
          acc_total = 0;
        
        step += 1
        offset += (n_input+1)

#### Executing LSTM Model
-----------


In [5]:
## logging results
log_path = '/output/tensorflow/logfile/'
writer = tf.summary.FileWriter(log_path)

## Load and prepare data
dh = DataHandler()
training_data = dh.read_data(text)
dic, rev_dic = dh.build_dataset(training_data)

## tensorflow graph
n_input = 3
n_units = 512

# creating placeholders(starting nodes of tensorflow computational graph) for x and y
x = tf.placeholder("float", [None,n_input,1])
y = tf.placeholder("float", [None,len(dic)])

# creating variables for RNN output weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_units,len(dic)]))
}
biases = {
    'out': tf.Variable(tf.random_normal([len(dic)]))
}

## create model
rnn_generator = RNNGenerator()
lstm = rnn_generator.create_LSTM(x,weights,biases,n_input,n_units)

# create loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=lstm, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(cost)

# model evaluation
correct_pred = tf.equal(tf.argmax(lstm,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

# initializing variables
initializer = tf.global_variables_initializer()

# run session
session_runner = SessionRunner(optimizer, accuracy, cost, lstm, initializer, writer)
session_runner.run_session(x,y,n_input,dic,rev_dic,training_data)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Iteration = 1000, Average Accuracy= 16.50%
Iteration = 2000, Average Accuracy= 46.50%
Iteration = 3000, Average Accuracy= 63.70%
Iteration = 4000, Average Accuracy= 57.40%
Iteration = 5000, Average Accuracy= 67.70%
Iteration = 6000, Average Accuracy= 68.60%
Iteration = 7000, Average Accuracy= 69.30%
Iteration = 8000, Average Accuracy= 74.40%
Iteration = 9000, Average Accuracy= 74.00%
Iteration = 10000, Average Accuracy= 82.50%
Iteration = 11000, Average Accuracy= 85.20%
Iteration = 12000, Average Accuracy= 84.40%
Iteration = 13000, Average Accuracy= 88.30%
Iteration = 14